In [310]:
import geopandas as gpd
import networkx as nx
import numpy as np
from libpysal import weights
import matplotlib.pyplot as plt
import momepy
from shapely.geometry import Polygon

In [311]:
def convert_polygons_to_centroids(gdf):
    gdf["polygons"] = gdf['geometry']
    gdf['geometry'] = gdf.geometry.centroid
    return gdf

In [312]:
def filter_points_based_on_specified_region(gdf, region):
    gdf = gdf[gdf.geometry.within(region)]
    return gdf

In [313]:
def convert_geojsons_with_lines_to_graph(gdf):
    G = momepy.gdf_to_nx(gdf, approach="primal", multigraph=False, directed=False)
    '''
    positions = {n: [n[0], n[1]] for n in list(G.nodes)}
    f, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
    gdf.plot(color="k", ax=ax[0])
    for i, facet in enumerate(ax):
        facet.set_title(("Улицы", "Graph")[i])
        facet.axis("off")
    nx.draw(G, positions, ax=ax[1], node_size=5)
    '''
    return G

In [314]:
def convert_geojsons_with_polygons_and_points_to_graph(gdf):
    coordinates = coordinates = np.column_stack((gdf.geometry.x, gdf.geometry.y))
    graph_init = weights.DistanceBand.from_array(coordinates, threshold=0.001)
    #graph_init = weights.KNN.from_dataframe(gdf, k=2)
    graph = graph_init.to_networkx()
    '''
    positions = dict(zip(graph.nodes, coordinates))
    f, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
    if "polygons" in gdf:
        gdf["polygons"].plot(color="k", ax=ax[0])
    else:
        gdf.plot(color="k", ax=ax[0])
    for i, facet in enumerate(ax):
        facet.set_title(("Строения/остановки", "Graph")[i])
        facet.axis("off")
    nx.draw(graph, positions, ax=ax[1], node_size=5)
    plt.show()
    '''
    return graph

In [ ]:
def visualise_final_graph(graph, objects_info):
    '''
    positions = dict(zip(graph.nodes, coordinates))
    f, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
    objects_info["polygons"].plot(color="k", ax=ax[0])
    objects_info["points"].plot(color="k", ax=ax[0])
    objects_info["lines"].plot(color="k", ax=ax[0])
    for i, facet in enumerate(ax):
        facet.set_title(("Строения/Остановки/Дороги", "Итоговый граф")[i])
        facet.axis("off")
    nx.draw(graph, positions, ax=ax[1], node_size=5)
    plt.show()
    '''

In [315]:
geojsons_with_polygons = ["Дома_исходные.geojson"]#, "House_1очередь_ЖК.geojson", "House_2очередь_ЖК.geojson", "House_3очередь_ЖК.geojson"]
geojsons_with_lines = ["Streets_исходные.geojson"]#, "Streets_1очередь.geojson", "Streets_2очередь.geojson", "Streets_3очередь.geojson"]
geojsons_with_points = ["Остановки_ОТ.geojson"]#, "Выходы_метро.geojson"]
# Обозначим границы региона
x_min, y_max = 37.472381, 55.551288 
x_max, y_min = 37.500082, 55.539195
region = Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)])

In [ ]:
graphs = {}
objects_info = {"polygons": [], "points": [], "lines": []}

for idx, file in enumerate(geojsons_with_points):
    gdf = gpd.read_file(f"./data/vector_data/{file}") 
    gdf["geometry"] = gdf["geometry"].to_crs("EPSG:4326")
    # Оставили точки из нужного региона
    gdf = filter_points_based_on_specified_region(gdf, region)
    graphs[f"p{idx+1}"] = convert_geojsons_with_polygons_and_points_to_graph(gdf)
    objects_info["points"].append(gdf)


for idx, (geojsons_with_line, geojsons_with_polygon) in enumerate(zip(geojsons_with_lines, geojsons_with_polygons)):
    # Обрабатываем дороги
    gdf = gpd.read_file(f"./data/vector_data/{geojsons_with_line}") 
    gdf["geometry"] = gdf["geometry"].to_crs("EPSG:4326")
    gdf = filter_points_based_on_specified_region(gdf, region)
    graphs[f"L{idx+1}"] = convert_geojsons_with_lines_to_graph(gdf)
    objects_info["lines"].append(gdf)

    # Обрабатываем строения
    gdf = gpd.read_file(f"./data/vector_data/{geojsons_with_polygon}") 
    gdf["geometry"] = gdf["geometry"].to_crs("EPSG:4326")
    gdf = convert_polygons_to_centroids(gdf)
    gdf = filter_points_based_on_specified_region(gdf, region)
    graphs[f"P{idx+1}"] = convert_geojsons_with_polygons_and_points_to_graph(gdf)
    objects_info["polygons"].append(gdf["polygons"])

result_graph = nx.union_all(graphs.values(), rename=graphs.keys())

d:\vscode\Kaliningrad_hack_2024\AI_working_tool_for_urban_development_data\env\Lib\site-packages\momepy\utils.py:352: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length
C:\Users\trans\AppData\Local\Temp\ipykernel_26496\1442734587.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf.geometry.centroid
d:\vscode\Kaliningrad_hack_2024\AI_working_tool_for_urban_development_data\env\Lib\site-packages\libpysal\weights\util.py:826: UserWarning: The weights matrix is not fully connected: 
 There are 17 disconnected components.
 There are 13 islands with ids: 1, 8, 9, 11, 15, 16, 201, 202, 205, 206, 208, 209, 210.
  w = W(neighbors, weights, ids, **kwargs)